# Relaxation Assignment
## The Stokes Flow

##### Laurent Pétré & Ilan Renous

In this assignment, we are going to simulate the interaction of two generic chemical species reacting. We are going tu use a two dimension version of a model called the Gray-Scott model. Let's get started!

In [ ]:
# We import the libraries we will need
import numpy
from matplotlib import pyplot, cm
%matplotlib inline

## The Stokes Flow

The Gray-Scott model represents the reaction and diffusion of two generic chemical species, $U$ and $V$, whose concentrations at a point in space are represented by variables $u$ and $v$. The model follows some simple rules.  

*  Each chemical _diffuses_ through space at its own rate.
*  Species $U$ is added at a constant feed rate into the system.
*  Two units of species V can 'turn' a unit of species U into V: $\; 2V+U\rightarrow 3V$
*  There's a constant kill rate removing species $V$.

This model results in the following system of partial differential equations for the concentrations $u(x,y,t)$ and $v(x,y,t)$ of both chemical species:
\begin{align}
\frac{\partial u}{\partial t} &= D_u \nabla ^2 u - uv^2 + F(1-u)\\
\frac{\partial v}{\partial t} &= D_v \nabla ^2 v + uv^2 - (F + k)v
\end{align}

with $D_u$ and $D_v$ the respective rates of diffusion

and $\nabla ^2$ is the Laplacian:

$$
\nabla ^2 u = \frac{\partial ^2 u}{\partial x^2} + \frac{\partial ^2 u}{\partial y^2}
$$

### Discretisation of the equations

We need to discretise these equations to get a numerical approximation. We are going to use a forward-time, centered-space discretization . For the first equation, this gives us:

$$
\frac{u^{n+1}_{i,j} - u^n_{i,j}}{\Delta t} =\\ D_u \left( \frac{u^n_{i+1, j} - 2u^n_{i,j} + u^n_{i-1,j}}{\Delta x^2} + \frac{u^n_{i, j+1} - 2u^n_{i,j} + u^n_{i,j-1}}{\Delta y^2}\right)+(G_1)^n_{i,j}
$$

Where $(G_1)^n_{i,j}=-u^n_{i,j}  (v^n_{i,j})^2+ F(1-u^n_{i,j})$

Rearranging the equation to solve for the value at the next time step, $u^{n+1}_{i,j}$, and assume that $\Delta x = \Delta y = \delta$, yields

$$
u^{n+1}_{i,j}=\\ u^n_{i,j} +\Delta t \left( \frac{D_u}{\delta ^2} \left( \ u^n_{i+1, j} - 4u^n_{i,j} + u^n_{i-1,j} + u^n_{i, j+1}+ u^n_{i,j-1}\right)+(G_1)^n_{i,j}\right)
$$

Similarly, for the second equation, we have 

$$
v^{n+1}_{i,j}= \\v^n_{i,j} +\Delta t \left( \frac{D_v}{\delta ^2} \left(v^n_{i+1, j} - 4v^n_{i,j} + v^n_{i-1,j} + v^n_{i, j+1} + v^n_{i,j-1}\right)+(G_2)^n_{i,j}\right)
$$

Where $(G_2)^n_{i,j}=u^n_{i,j}  (v^n_{i,j})^2- (F+k)v^n_{i,j}$

That's a little messy, but we will have all the information we need.

## Boundary conditions

For this problem, we are going to use zero Neumann boundary conditions on all sides of the domain. These are : 

$$
\frac{\partial u}{\partial x}= \frac{u^n_{N,j} - u^n_{N-1,j}}{\delta} =0
$$

$$
\frac{\partial u}{\partial y}= \frac{u^n_{i,N} - u^n_{i,N-1}}{\delta} =0
$$

$$
\frac{\partial v}{\partial x}= \frac{v^n_{N,j} - v^n_{N-1,j}}{\delta} =0
$$

$$
\frac{\partial v}{\partial y}= \frac{v^n_{i,N} - v^n_{i,N-1}}{\delta} =0
$$

Where N is the index of the cell at the boundaries of our spacial domain (i.e. N = 0 and N = 192 here).

## Implementation

We can now implement those equations. The boundary conditions need to be applied at each itterations. In Python that gives us :

In [ ]:
def solve(psi_i, omega_i, dx, dy, norm_func, norm_target):
    """
    Compute the solution of the partial differential system : 
        - du/dt = Du*(div(grad(u)) + G1(u, v)
        - dv/dt = Dv*(div(grad(v)) + G2(u, v).

    Parameters
    ----------
    psi_i, omega_i: array[NxN] of floats
        initial conditions

    dx, dy: float
        space steps

    nt: integer
        number of iterations

    norm_func: function of 2 arrays[NxN] of floats
        computes the norm between two iterations

    norm_target: float
        norm between two iteration needed to stop the computation

    Returns
    -------
    psi, omega: array[NxN]
        solution of the system
    """
    
    # We don't want to modify the input parameters
    psi = psi_i.copy()
    omega = omega_i.copy()

    # Initial norms for psi and omega
    normp = 1
    normo = 1

    while ((normp > norm_target) or (normo > norm_target)):
        # Omega computation
        omegan = omega.copy()

        omega[1:-1,1:-1] = .25 * (omegan[1:-1,2:] + omegan[1:-1, :-2] \
                + omegan[2:, 1:-1] + omegan[:-2, 1:-1])

        # Bcs
        omega[-1, :] = (-1/(2*dy**2))*(8*psi[-2, :] - psi[-3, :]) \
                - (3*1)/dy # Top
        omega[0, :] = (-1/(2*dy**2))*(8*psi[1, :] - psi[2, :]) # Bottom
        omega[:, -1] = (-1/(2*dx**2))*(8*psi[:, -2] - psi[:, -3]) # Left
        omega[:, 0] = (-1/(2*dx**2))*(8*psi[:, 1] - psi[:, 2]) # Right

        # Psi computation
        psid = psi.copy()

        psi[1:-1,1:-1] = 1/(2*(dx**2 + dy**2)) * ( \
                (psid[1:-1,2:]+psid[1:-1,:-2])*dy**2 + \
                (psid[2:,1:-1] + psid[:-2,1:-1])*dx**2 + \
                omega[1:-1,1:-1]*dx**2*dy**2)

        # BCs automatically satisfied

        normp = norm_func(psid, psi)
        normo = norm_func(omega, omegan)

    return psi, omega

## Solve !

We created the function needed to solve this problem, all we need now is initial conditions and simulation parameters. Let's choose the following parameters :

* Grid of points with dimension `192x192` points
* Domain is $5{\rm m} \times 5{\rm m}$
* Final time is $8000{\rm s}$
* $Du, Dv, F, k = 0.00016, 0.00008, 0.035, 0.065 $
* For the initial distribution of $u$ and $v$, we download a grid given in the lesson.

We saw that, in two dimension, the stability constraints for an explicit scheme is given by

$$
\alpha \frac{\Delta t}{(\Delta x)^2} + \alpha \frac{\Delta t}{(\Delta y)^2} < \frac{1}{2}.
$$

With $\Delta x = \Delta y = \delta$ the stability condition is:

$$
\alpha \frac{\Delta t}{\delta^2} < \frac{1}{4}
$$

To be sure to stay within the stability condition, we set our time step to be 

$$\Delta t = \frac{9}{40}\frac{\delta^2}{\max(D_u, D_v)}$$

In [ ]:
# Space conditions
nx = 41
ny = 41

l = 1.
h = 1.

dx = l/(nx-1)
dy = h/(ny-1)

# Convergence conditions
def l1_norm(new, old):
    return numpy.sum(numpy.abs(new - old))

norm_target = 1e-6

# Initial conditions
psi = numpy.zeros((nx, ny))
omega = numpy.zeros((nx, ny))

Now, it's time to launch the simulation :

In [ ]:
# Solve
psi, omega = solve(psi, omega, dx, dy, l1_norm, norm_target)

# Output
print(numpy.amax(numpy.abs(psi)))
print(numpy.amax(numpy.abs(omega)))
print(numpy.round(psi[32,::8], 4))

## Drawing

Thanks to this animation, we can see the evolution of the concentration of our species. The high concentration zones are represented by dots.

In [ ]:
# Axes helpers
x = numpy.linspace(0, l, nx)
y = numpy.linspace(0, h, ny)

# Figure
pyplot.figure(figsize=(15,15))
pyplot.contourf(x,y,psi,10,cmap=cm.viridis)

## Conclusion

Using a explicit scheme was easier to implement in code but we had to take a very small time step to reach stability. Implicit scheme would not have required such small time step to give a stable approximation.

It was intersting to change the values of the different parameters to see different patterns of density evovle in time.

##### Source

(1) We used the following lectures https://github.com/numerical-mooc/numerical-mooc available under Creative Commons Attribution license CC-BY 4.0, (c)2014 L.A. Barba, C. Cooper, G.F. Forsyth, A. Krishnan.

---
###### The cell below loads the style of this notebook. 

In [ ]:
# Execute this cell to load the notebook's style sheet, then ignore it
from IPython.core.display import HTML
css_file = '../../styles/numericalmoocstyle.css'
HTML(open(css_file, "r").read())